# visualize processing stages

Notebook to plot multiple mass channels at different stages of low-level processing for each point in each given slide.  
Each mass channel can be plotted against the background using red/blue comparison (bg in <span style="color:red;text-shadow:0px 0px 2px white">__red__</span>, mass channel in <span style="color:cyan;text-shadow:0px 0px 2px black">__blue__</span>, overlap in <span style="color:white;text-shadow:0px 0px 2px black">__white__</span> ).  
Images are saved as `.png` files within a directory structure headed at input `save_path`

Accepted format for input images: IonPath's __MIBITiff__ format from MIBI/O or MIBITracker

Please update the __input parameters__ as needed

## load headers

In [1]:
# magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2

#imports
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pathlib
import glob
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mibidata import mibi_image as mi, tiff
import visualize_data as viz

## input parameters

In [2]:
data_path = pathlib.Path('/Volumes/DISK_IMG/')
save_path = pathlib.Path('/Users/adam_kagel/Desktop/comp/')

mass_channels = [89, 113, 146, 158, 175, 176, 115]

# background channel to compare to.  For no background comparison, set to -1
bg_channel = 197

# Option for comparing to bg channel
# pixels w/ non zero values -> 1.0 (shows all overlaps equally)
binary = False

# Anonymized by default currently
# TODO: Read panel if anonymize is false, and use target names
#anonymize_targets = True
#panel_path = data_path.joinpath('Panel54_MelanomaCohort_2.csv')

slide_foldernames = [
    '191204_CohortSlide9_S14-47859-A2-S14-20352-A8',
    '191205_CohortSlide7_S146936A2_G1471023A1_run',
    '191206_CohortSlide11_TMA443_run',
    '191211_CohortSlide10_S158066A1_SP153393_G1675892_run'
]

# leave empty to use full slide name
slide_names = [
    'S9',
    'S7',
    'S11',
    'S10'
]

process_folder_prefix = ''
process_folder_suffix = '_TIFF'

raw_tiff_folder_name = 'bg_none'
proccessed_tiff_folder_name = 'bg_au_050_ta_020'

# process 'prefix' is PointN
process_suffix = '_RowNumber0_Depth_Profile0'
isobar_suffix = '-MassCorrected'
denoise_suffix = '-Filtered'

# Show only given mass channels
show_raw = mass_channels
show_bg_cor = mass_channels
show_iso = [115]
show_filter = mass_channels

## plot images

In [6]:
plt.ioff()
plt.style.use('dark_background')

# show matrix
show_m = np.zeros((4,len(mass_channels)))
show_m[0,:] = np.isin(mass_channels, show_raw)
show_m[1,:] = np.isin(mass_channels, show_bg_cor)
show_m[2,:] = np.isin(mass_channels, show_iso)
show_m[3,:] = np.isin(mass_channels, show_filter)
show_m = np.ma.make_mask(show_m, shrink=False)

def plot_text(ax,text,fontsize):
    ax.text(0.5,0.5,text,verticalalignment='center',horizontalalignment='center',transform=ax.transAxes, color='yellow', fontsize=fontsize)
    ax.set_frame_on(False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

def make_fig(tiffs,slide,point,channel,titles):
    ncols = len(tiffs)+1
    
    plt.subplots_adjust(hspace=0.0,wspace=0.0)
    wr = [1] + ([3] * (ncols-1))
    gs = gridspec.GridSpec(2,ncols,height_ratios = [1,3],width_ratios=wr, hspace=0)
    
    for n, cell in enumerate(gs):
        ax = plt.subplot(cell)
        if n//ncols == 0:
            plot_text(ax,titles[n],14)
        else:
            if n%ncols==0:
                plot_text(ax,str(channel),14)
            else:
                image = tiff.read(str(tiffs[n%ncols-1]))
                im = image[channel]
                if bg_channel<0:
                    viz.plot_image(im, ax=ax,brighten_image=True,hi_res=True)
                else:
                    viz.plot_rb_image(image[bg_channel],im,ax=ax,binary=binary,normalize=True,brighten_image=True,hi_res=True)
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
    
    '''
    # TODO: too many if/elif/else's
    for n, cell in enumerate(gs):
        ax = plt.subplot(cell)
        if n//ncols == 0:
            plot_text(ax,titles[n],14)
        elif n//ncols == 1:
            if n%ncols ==0:
                plot_text(ax,str(channel),14)
            else:
                image = tiff.read(str(tiffs[n%ncols-1]))
                im = image[channel]
                viz.plot_image(im, ax=ax, brighten_image=True, hi_res=True)
        else:
            if n%ncols == 0:
                plot_text(ax,str(bg_channel),14)
            elif n%ncols == 1:
                image = tiff.read(str(tiffs[0]))
                im = image[bg_channel]
                viz.plot_image(im, ax=ax, brighten_image=True, hi_res=True)
            else:
                ax.axis('off')
        
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
    '''
    if not save_path.is_dir():
        save_path.mkdir()
    comp_dir_name = f'{slide}-{point}'
    comp_dir_path = save_path.joinpath(comp_dir_name)
    if not comp_dir_path.is_dir():
        comp_dir_path.mkdir()
    
    plt.gcf().set_figheight((1024*4/3)/300)
    plt.gcf().set_figwidth((1024*(ncols-2/3))/300)
    
    # save as .png for lossless compression
    plt.subplots_adjust(hspace=0.0,wspace=0.0)
    plt.savefig(str(comp_dir_path.joinpath(f'{channel}.png')), bbox_inches='tight', dpi=300, pad_inches=0.0)
    
    

for slide_n, slide in enumerate(slide_foldernames):
    tiffpath = data_path.joinpath(slide).joinpath(process_folder_prefix+slide+process_folder_suffix)
    rawpath = tiffpath.joinpath(raw_tiff_folder_name)
    #print(slide)
    for rawfilename in glob.glob(f'{str(rawpath)}/*{process_suffix}.tiff'):
        pointNum = int(re.search(r'\d+',pathlib.Path(rawfilename).name).group())
        
        proc_tiffpath = tiffpath.joinpath(proccessed_tiff_folder_name)
        
        all_tiffs = np.array([
            rawpath.joinpath(rawfilename),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}.tiff'),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}{isobar_suffix}.tiff'),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}{isobar_suffix}{denoise_suffix}.tiff')
        ])
        
        all_titles = np.array([
            'Raw Image',
            'Background Removed',
            'Isobaric Correction',
            'Denoised'
        ])
        
        # TODO: programatically select which tiffs to plot for each channel, from input parameters
        for n_chan, channel in enumerate(mass_channels):
            #print(all_tiffs)
            #print(show_m[:,n_chan])
            #titles = all_titles[show_m[:,n_chan]]
            #print(np.concatenate((['Mass Channel'], all_titles[show_m[:,n_chan]])))
            make_fig(all_tiffs[show_m[:,n_chan]],slide_names[slide_n],pointNum,channel,np.concatenate((['Mass Channel'], all_titles[show_m[:,n_chan]])))
            
            '''
            if channel != 115:
                make_fig(tiffs,slide_names[slide_n],pointNum,channel,titles1)
            else:
                tiffs2 = tiffs.copy()
                tiffs2.insert(2,proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}{isobar_suffix}.tiff'))
                make_fig(tiffs2,slide_names[slide_n],pointNum,channel,titles2)
            #break #debug
            '''
            #break #debug
            
        #break #debug
    #break #debug

counts: 111241.03848330495
counts: 111264.23385167464
counts: 104715.812799043
counts: 72829.14168247953
counts: 72557.24933155073
counts: 45757.36697860968
counts: 12557.45688780686
counts: 12789.073970840476
counts: 11829.489065180105
counts: 14516.468982630266
counts: 14728.525349650323
counts: 12660.474067599078
counts: 39497.14168247947
counts: 39254.77874331553
counts: 27399.837566844923
counts: 66906.66129032255
counts: 66351.93560606064
counts: 44724.76893939398
counts: 30131.094623655914
counts: 29968.93560606062


/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "fov_id" attribute is required if "folder" is specified. Setting "fov_id" to Point2-MassCorrected.
  f'The "{field_name}" attribute is required if "{value_name}" '
/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "folder" attribute is required if "fov_id" is specified. Setting "folder" to Point2-MassCorrected.
  f'The "{field_name}" attribute is required if "{value_name}" '


counts: 29830.491161616173
counts: 21377.557828282825


/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "fov_id" attribute is required if "folder" is specified. Setting "fov_id" to Point4.
  f'The "{field_name}" attribute is required if "{value_name}" '
/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "folder" attribute is required if "fov_id" is specified. Setting "folder" to Point4.
  f'The "{field_name}" attribute is required if "{value_name}" '


counts: 8536.166666666666
counts: 8536.166666666666


...

counts: 178070.47731064394
counts: 172162.73163548158


/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "fov_id" attribute is required if "folder" is specified. Setting "fov_id" to Point3-MassCorrected-Filtered.
  f'The "{field_name}" attribute is required if "{value_name}" '
/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "folder" attribute is required if "fov_id" is specified. Setting "folder" to Point3-MassCorrected-Filtered.
  f'The "{field_name}" attribute is required if "{value_name}" '


counts: 169229.40731115727
counts: 180081.10493827157
counts: 170812.98789173784
counts: 163561.43233618213
counts: 148746.08271604934
counts: 141763.33974358995
counts: 136701.83974358987
counts: 141634.6427160494
counts: 136590.5997435898
counts: 135314.3597435898
counts: 178773.56564287853
counts: 174737.97487872464
counts: 170783.97487872478
counts: 182357.38271604924
counts: 177025.41117216105
counts: 165115.9826007325
counts: 150444.21604938264
counts: 141445.73974358971


/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "fov_id" attribute is required if "folder" is specified. Setting "fov_id" to Point3-MassCorrected.
  f'The "{field_name}" attribute is required if "{value_name}" '
/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "folder" attribute is required if "fov_id" is specified. Setting "folder" to Point3-MassCorrected.
  f'The "{field_name}" attribute is required if "{value_name}" '


counts: 139478.50076053894
counts: 137108.02618426768
counts: 94279.1483781918
counts: 93667.9449472096
counts: 90725.10834590254
counts: 68332.98726708078
counts: 67359.10769230763
counts: 58688.107692307734
counts: 20080.28726708075
counts: 19723.607692307687
counts: 13887.20769230769
counts: 16870.515527950294
counts: 16342.807692307699
counts: 13968.19899665552
counts: 88857.09609061014
counts: 88023.78808446457
counts: 81425.90573152335
counts: 53453.18541522882
counts: 52309.95584045581
counts: 45996.51139601139
counts: 35428.037267080756
counts: 34059.99413298567
counts: 33841.75684485008
counts: 26896.502607561924
